# Oppgave
En kjøpesenterkjede har leid inn BearingPoint fordi de ønsker å bli mer datadrevet. Som et steg i denne prosessen har vi identifisert at det kan være nyttig å segmentere kundene for å skreddersy kommunikasjonen til kundene i større grad.

I denne Notebooken finner dere et konstruert datasett med 5 dimensjoner: `CustomerID`,	`Gender`,	`Age`,	`Annual Income (k$)` og	`Spending Score (1-100)`
Det er allerede gjort en god del "Exploratory Data Analysis" (EDA) som dere kan se på for å forstå dataene, dere står fritt til å grave mer om det skulle være nødvendig.
Under EDA har vi laget noen funksjoner som bør være nyttige for denne oppgaven. Det er også en del nyttige kommentarer rundt omkring i Notebooken som forklarer plot og funksjoner. Det anbefales å se på disse. 

1. Dere skal gjøre en cluster analyse med Kmeans clustering. Det er opp til dere å velge hvilke dimensjoner og hvor mange cluster som er riktig. Hint: bruk Elbow method til å finne riktig antall cluster når dere gjennomfører analysen.
2. Ledelsen i kjøpesenterkjeden har sett at deres tidligere kampanjer har hatt lite effekt på omsetningen. De har en hypotese om at dette kan skyldes at de tidligere ikke har truffet de riktige segmentene. Ledelsen ønsker at dere definerer segmenter basert på datagrunnlaget i denne Notebooken og presenterer tiltak som vil kunne øke omsetningen basert på disse segmentene.

PS: Det er ikke behov for å lage en egen PowerPoint for å presentere, her kan dere bare ta utgangspunkt i Notebooken.

# K-Means Clustering
K-means clustering er en type unsupervised Machine Learning-algoritme (så der har dere noen buzzwords å snakke om), som brukes til å gruppere data som ligner på hverandre over én eller flere (potensielt veldig mange) dimensjoner eller variable. Clustering er at man deler dataen inn i grupper, clusters, og K-means er en algoritme for å gjøre dette basert på euklidisk distanse mellom dataene. Dere trenger ikke forstå eller implementere den spesifikke algoritmen i denne oppgaven. I K-means clustering må man velge antallet clusters man vil dele dataen inn i, K, før algoritmen kjøres. Det er derfor vanlig å kjøre algoritmen for flere K og så velge det antallet som gir best resultat, basert på et eller annet måltall. Det er her Elbow method kommer inn. Man må også velge hvilke dimensjoner man skal ta med i clusteringen. Man bruker derfor en del tid på å analysere og gjøre seg kjent med dataen på forhånd, slik at man for eksempel kan finne ut om det er noen dimensjoner som ikke er relevante.

Dere kan lese mer om K-means clustering for eksempel her: 

https://towardsdatascience.com/understanding-k-means-clustering-in-machine-learning-6a6e67336aa1

https://www.geeksforgeeks.org/k-means-clustering-introduction/

Og om Elbow method her: 

https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/

http://www.nbertagnolli.com/jekyll/update/2015/12/10/Elbow.html

<img src="https://images.unsplash.com/photo-1519567241046-7f570eee3ce6?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1000&q=80" width="1000px">

# Lese inn data og installere nødvendige bibliotek

**Installerer bibliotek**

In [ ]:
# for basic mathematics operation 
import numpy as np
import pandas as pd
from pandas import plotting

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# for interactive visualizations
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff


from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# for path
import os
print(os.listdir('../input/'))

**Leser inn datasett**
Her ser du også hva de forskjellige dimensjonene heter

In [ ]:
# importing the dataset
data = pd.read_csv('../input/Mall_Customers.csv')

data.head()

**Litt enkel informasjon om dataen**

In [ ]:
# describing the data

data.describe()

In [ ]:
# checking if there is any NULL data

data.isnull().sum()

# Datavisualisering

## Visualisering av dimensjoner hver for seg

Hvilke dimensjoner vil intuitivt være viktige for kjøpesenteret å kjenne til når de skal tilpasse salgskommunikasjonen sin til kundene?

**Empirisk fordeling**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (25, 8)

plt.subplot(1, 3, 1)
sns.set(style = 'whitegrid')
sns.distplot(data['Annual Income (k$)'])
plt.title('Distribution of Annual Income', fontsize = 20)
plt.xlabel('Range of Annual Income')
plt.ylabel('Count')


plt.subplot(1, 3, 2)
sns.set(style = 'whitegrid')
sns.distplot(data['Age'], color = 'red')
plt.title('Distribution of Age', fontsize = 20)
plt.xlabel('Range of Age')
plt.ylabel('Count')


plt.subplot(1, 3, 3)
sns.set(style = 'whitegrid')
sns.distplot(data['Spending Score (1-100)'], color = 'green')
plt.title('Distribution of Spending Score', fontsize = 20)
plt.xlabel('Range of Spending Score')
plt.ylabel('Count')
plt.show()

> I plottene over ser man fordelingen av inntekt, alder og spending score for kundene. Her kan man notere seg noen initielle trender og typiske trekk ved kunder. Hvem er kundene som handler?


**Mer detaljert oversikt**

In [ ]:
data['Gender'].value_counts().plot(kind='bar')


> Her ser man antall kunder som er kvinner og menn.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
sns.countplot(data['Age'], palette = 'hsv')
plt.title('Distribution of Age', fontsize = 20)
plt.show()

> Fordeling i antall kunder av hver alder fra 18 til 70 år.

In [ ]:
plt.rcParams['figure.figsize'] = (20, 8)
sns.countplot(data['Annual Income (k$)'], palette = 'rainbow')
plt.title('Distribution of Annual Income', fontsize = 20)
plt.show()

> Detaljert oversikt over hvilken inntekt kunder har. Hvilket inntektsområde er mest vanlig?

In [ ]:
plt.rcParams['figure.figsize'] = (20, 8)
sns.countplot(data['Spending Score (1-100)'], palette = 'copper')
plt.title('Distribution of Spending Score', fontsize = 20)
plt.show()

> Detaljert fordeling av spending scores. 

## Sammenheng mellom dimensjoner

Når man driver analyse på data, er det viktig å se på interaksjonen mellom dimensjonene man har. Vi skal gruppere kunder på tvers av flere dimensjoner, og det er da lurt å ha innsikt i om det er noen sammenheng mellom dimensjonene. Er det noen sett med dimensjoner som i større grad bidrar til å segmentere kundene enn andre? 

Her er et pairplot av dimensjonene, det vil si at kundene er plottet i et grid på to dimensjoner om gangen. Man kan da se om det er noen sammenhenger mellom disse dimensjonene. Er det for eksempel sånn at noen aldersgrupper tjener mer enn andre, eller bruker mer penger enn andre? Ser det ut til at hvor mye penger man bruker og hvor mye man tjener har en sammenheng? Er det noen dimensjonskombinasjoner som ser ut til å dele kundene i grupper mer enn andre? Kanskje er det også noen litt merkelige sammenhenger innimellom (en konsekvens av konstruert datasett?)? Noen tips til hvordan lese grafene finnes under figuren. 

In [ ]:
#Create a numberic label column for gender:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['Gender_Numeric'] = label_encoder.fit_transform(data['Gender'])

In [ ]:
sns.pairplot(data)
plt.title('Pairplot for the Data', fontsize = 20)
plt.show()

> Figuren kan se litt rotete ut, men se på hver enkelt firkant for seg. På diagonalen finner du plottene vi allerede har sett på, nemlig hver dimensjon hver for seg. Du finner de samme plottene på hver side av diagonalen, men med aksene snudd. I disse har du én dimensjon på hver akse, og scorene til en enkelt kunde ses som en blå prikk.

**Korrelasjon**

Korrelasjon er hvor mye to dimensjoner, eller variable, henger sammen. Korrelasjonen går fra -1 til 1, hvor begge ytterpunktene betyr at variablene er perfekt korrelerte. Positiv korrelasjon vil si at høye verdier i den ene variabelen har en tendens til å gi høye verdier i den andre, mens høye verdier i én variabel gir ofte lave i den andre dersom de to er negativt korrelerte. Ved 0 korrelasjon er det ingen sammenheng mellom verdiene i den ene og andre variabelen. 

Hvorfor er korrelasjon relevant? Å bruke sterkt korrelerte variable i analyse, kan gi misvisende resultater. Det blir vanskelig å trekke konklusjoner om kausalitet (Dersom eldre mennesker har en tendens til å ha diabetes og svake lunger, er det da alder, diabetes eller svake lunger som gjør at disse er i risikogruppen for alvorlig covid-19?). I tillegg vil to sterkt korrelerte variable i stor grad representere det samme konseptet, og dersom begge variablene tas med i analysen, vil dette konseptet telle dobbelt i algoritmen. Man bør derfor vurdere å droppe en dimensjon dersom det er veldig sterk korrelasjonen (enten positiv eller negativ) mellom denne og en annen dimensjon.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
sns.heatmap(data.corr(), cmap = 'Wistia', annot = True)
plt.title('Heatmap for the Data', fontsize = 20)
plt.show()

> I denne grafen er korrelasjonen mellom de forskjellige dimensjonene notert. Grafen leses tilsvarende parplottet over. Du kan se bort fra diagonalen når du ser etter korrelerte dimensjoner, ettersom diagonalen viser korrelasjon mellom en dimensjon og seg selv, som alltid er 1. 

### Flere plot for forskjell mellom mann og kvinne

In [ ]:
#  Gender vs Spendscore

plt.rcParams['figure.figsize'] = (18, 7)
sns.boxenplot(data['Gender'], data['Spending Score (1-100)'], palette = 'Blues')
plt.title('Gender vs Spending Score', fontsize = 20)
plt.show()

>Plottet viser hvilke Spending Scores som er mest vanlig for menn og kvinner. Særlig de store boksene på midten forteller henholdsvis hvilke scores en mann og en kvinne typisk har. Er det en vesentlig forskjell?

In [ ]:
plt.rcParams['figure.figsize'] = (18, 7)
sns.violinplot(data['Gender'], data['Annual Income (k$)'], palette = 'rainbow')
plt.title('Gender vs Annual Income', fontsize = 20)
plt.show()

> En annen type plot (er bare til å bytte dette til "boxenplot" som over om du ønsker) for årlig inntekt per kjønn. 

In [ ]:
plt.rcParams['figure.figsize'] = (18, 7)
sns.stripplot(data['Gender'], data['Age'], palette = 'Purples', size = 10)
plt.title('Gender vs Age', fontsize = 20)
plt.show()

> Nok en måte å visualisere kunder, her for alder per kjønn. Det ser ikke ut til å være en stor forskjell i alderfordelingen for menn og kvinner. Generelt ville man vel også synes det var litt spesielt om kvinner hadde en tendens til å være yngre eller eldre enn menn? Men i et datasett på 200 shoppere kunne man fått en forskjell. Det er da viktig å være litt kritisk til de funnene man gjør, og reflektere rundt hva som kan være grunnen.

# Nyttige funksjoner

In [ ]:
def run_elbow_method_to_find_optimal_no_of_clusters(x):
    from sklearn.cluster import KMeans

    wcss = []
    for i in range(1, 11):
        km = KMeans(n_clusters = i, init = 'k-means++')
        km.fit(x)
        wcss.append(km.inertia_)

    plt.plot(range(1, 11), wcss)
    plt.title('The Elbow Method', fontsize = 20)
    plt.xlabel('No. of Clusters')
    plt.ylabel('wcss')
    plt.show()

In [ ]:

import matplotlib
def fit_predict_and_visualize_in_2d_clusters(kmeans, x):
    ### Takes as input a sklearn kmeans model and a 2d matrix and plots the clusters###
    
    if x.shape[1] != 2:
        raise TypeError("This function only accepts matricies with 2 dimensions")
    
    if isinstance(x, pd.core.frame.DataFrame):
        columns = x.columns.to_list()
        x = x.values
        
    color_iterator = matplotlib.colors.cnames.__iter__()
    
    y_means = kmeans.fit_predict(x)
    
    for i in range(kmeans.get_params()['n_clusters']):
        plt.scatter(x[y_means == i, 0], x[y_means == i, 1], s = 100, c = next(color_iterator), label = f'Segment {i}')
        
    plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1], s = 50, c = 'blue' , label = 'centeroid')
    plt.style.use('fivethirtyeight')
    plt.title('K Means Clustering', fontsize = 20)
    if columns:
        plt.xlabel(columns[0])
        plt.ylabel(columns[1])
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def fit_predict_and_visualize_in_3d_clusters(kmeans, x):
    ### Takes as input a sklearn kmeans model and a 3d matrix and plots the clusters###
    
    if x.shape[1] != 3:
        raise TypeError("This function only accepts matricies with 3 dimensions")
        
    kmeans.fit(x)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    data = x
    data['labels'] =  labels
    trace1 = go.Scatter3d(
        x= data.iloc[:,0],
        y= data.iloc[:,1],
        z= data.iloc[:,2],
        mode='markers',
         marker=dict(
            color = data['labels'], 
            size= 10,
            line=dict(
                color= data['labels'],
                width= 12
            ),
            opacity=0.8
         )
    )
    df = [trace1]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0  
        ),
        scene = dict(
                xaxis = dict(title  = data.columns[0]),
                yaxis = dict(title  = data.columns[1]),
                zaxis = dict(title  = data.columns[2])
            )
    )

    fig = go.Figure(data = df, layout = layout)
    py.iplot(fig)

In [ ]:
# NOTE: It is not good design to take a k-means model and the whole dataframe as
# parameters when we only need the cluster centers and the column names, 
# but we wanted to keep the parameters similar between all visialization functions

def fit_predict_and_visualize_cluster_with_barchart(kmeans, x_dataframe):
    y_means = km.fit_predict(x_dataframe)
    df = pd.DataFrame(kmeans.cluster_centers_, columns= x_dataframe.columns)
    df = pd.DataFrame(data=StandardScaler().fit_transform(X=df), columns=df.columns)
    df.plot.bar(figsize=(12,7))
    plt.legend(bbox_to_anchor=(1,.6))
    plt.axhline(color = 'lightgrey', linestyle = '--')
    plt.xlabel('Cluster')

# Start oppgaven her:
Finn passende dimensjoner å segmentere dataene på og et passende antall cluster. Ta gjerne utgangspunkt i vurderinger dere har gjort i dataanalysen, men dere må nok også prøve dere fram og se på resultatene.
Eksempelkoden lager 2 cluster basert på dimensjonene `Age` og `Gender_Numeric`
Dere kan bruke så mange dimensjoner dere ønsker, men husk at dere skal klare å kommunisere hva segmentene innebærer.

In [ ]:
# TODO: Her velger dere hvilke features som skal være med clusteringen, skriv data.columns for å se alle kolonnene

features = ['Age','Gender_Numeric']
x = data[features]

In [ ]:

#Plotter WCSS mot antall cluster
#WCSS = the sum of squares of the distances of each data point in all clusters to their respective centroids
run_elbow_method_to_find_optimal_no_of_clusters(x)

In [ ]:
#TODO: Her må dere definere hvor mange clustere K-means skal segmentere i.
km = KMeans(n_clusters = 2, init = 'k-means++')
#2d-funksjonen kan kun brukes til å visualisere to dimensjoner, men kan byttes til 3d om man bruker tre dimensjoner.
fit_predict_and_visualize_in_2d_clusters(km, x)
# fit_predict_and_visualize_in_3d_clusters(km, x) #Kan brukes hvis dere har 3 features/dimensjoner som dere lager cluster basert på

> Segmenter i forskjellige farger. Blå prikk er sentrum av segmentet

In [ ]:
fit_predict_and_visualize_cluster_with_barchart(km, x)

> Dette plottet visere tendensene i verdiene til dimensjonene innad i hvert cluster. Verdiene er sentrert rundt 0, så søylene forteller om verdiene til dimensjonen i clusteret er lave, høye eller gjennomsnittlige. 

<img src="https://media1.tenor.com/images/251da9c5c43d4bf2e4ddb36ed6d4c2eb/tenor.gif?itemid=10603325" width="500px">